In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [2]:
# Generaset Dataset Dummy
np.random.seed(42)
n_samples = 5000

umur_kendaraan = np.random.randint(1, 15, size=n_samples)
harga_kendaraan = np.random.randint(50_000_000, 500_000_000, size=n_samples)
pengemudi_muda = np.random.choice([0, 1], size=n_samples)  # 1: <25 tahun
area_risiko = np.random.choice([0, 1, 2], size=n_samples)  # 0: aman, 1: sedang, 2: rawan

In [3]:
# Membuat Label Risiko dan Premi
risk_score = (
    (umur_kendaraan > 10).astype(int) +
    (harga_kendaraan < 150_000_000).astype(int) +
    pengemudi_muda +
    area_risiko
)
risk_label = pd.cut(risk_score, bins=[-1, 1, 3, 5], labels=[0, 1, 2]).astype(int)

base_premi = 150_000
premi = base_premi + (risk_label * 125_000) + (harga_kendaraan * 0.0035)
premi = premi.astype(int)


In [4]:
#  Membuat DataFrame dan Membagi Data
df = pd.DataFrame({
    'umur_kendaraan': umur_kendaraan,
    'harga_kendaraan': harga_kendaraan,
    'pengemudi_muda': pengemudi_muda,
    'area_risiko': area_risiko,
    'risk_label': risk_label,
    'premi': premi
})
features = df[['umur_kendaraan', 'harga_kendaraan', 'pengemudi_muda', 'area_risiko']]
target_risk = df['risk_label']
target_premi = df['premi']

X_train, X_test, y_risk_train, y_risk_test, y_premi_train, y_premi_test = train_test_split(
    features, target_risk, target_premi, test_size=0.2, random_state=42
)

In [5]:
# Standarisasi Fitur Numerik
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()
X_train_scaled[['umur_kendaraan', 'harga_kendaraan']] = scaler.fit_transform(X_train[['umur_kendaraan', 'harga_kendaraan']])
X_test_scaled[['umur_kendaraan', 'harga_kendaraan']] = scaler.transform(X_test[['umur_kendaraan', 'harga_kendaraan']])

In [6]:
# Membangun Model TensorFlow
input_layer = tf.keras.Input(shape=(4,))
x = tf.keras.layers.Dense(64, activation='relu')(input_layer)
x = tf.keras.layers.Dense(32, activation='relu')(x)

risk_output = tf.keras.layers.Dense(3, activation='softmax', name='risk_label')(x)
premi_output = tf.keras.layers.Dense(1, activation='linear', name='premi')(x)

model = tf.keras.Model(inputs=input_layer, outputs=[risk_output, premi_output])
model.compile(
    optimizer='adam',
    loss={
        'risk_label': 'sparse_categorical_crossentropy',
        'premi': 'mse'
    },
    metrics={
        'risk_label': 'accuracy',
        'premi': 'mae'
    }
)

In [7]:
# Melatih Model
history = model.fit(
    X_train_scaled,
    {'risk_label': y_risk_train, 'premi': y_premi_train},
    validation_data=(X_test_scaled, {'risk_label': y_risk_test, 'premi': y_premi_test}),
    epochs=30,
    batch_size=32,
    verbose=2
)

Epoch 1/30
125/125 - 5s - 38ms/step - loss: 1652452229120.0000 - premi_loss: 1652452229120.0000 - premi_mae: 1208980.1250 - risk_label_accuracy: 0.4683 - risk_label_loss: 1.0542 - val_loss: 1646741159936.0000 - val_premi_loss: 1643753897984.0000 - val_premi_mae: 1206026.6250 - val_risk_label_accuracy: 0.5750 - val_risk_label_loss: 0.9176
Epoch 2/30
125/125 - 1s - 9ms/step - loss: 1652182351872.0000 - premi_loss: 1652182351872.0000 - premi_mae: 1208877.6250 - risk_label_accuracy: 0.5698 - risk_label_loss: 0.9369 - val_loss: 1646161297408.0000 - val_premi_loss: 1643176132608.0000 - val_premi_mae: 1205808.6250 - val_risk_label_accuracy: 0.5840 - val_risk_label_loss: 0.8891
Epoch 3/30
125/125 - 1s - 7ms/step - loss: 1651067584512.0000 - premi_loss: 1651067584512.0000 - premi_mae: 1208464.7500 - risk_label_accuracy: 0.5845 - risk_label_loss: 0.9703 - val_loss: 1644408340480.0000 - val_premi_loss: 1641429860352.0000 - val_premi_mae: 1205160.7500 - val_risk_label_accuracy: 0.6610 - val_risk_l

In [8]:
# Evaluasi Model
eval_results = model.evaluate(X_test_scaled, {'risk_label': y_risk_test, 'premi': y_premi_test}, verbose=2)
print(f"Test Loss dan Metrics: {eval_results}")

32/32 - 0s - 7ms/step - loss: 587246010368.0000 - premi_loss: 588196020224.0000 - premi_mae: 708469.6875 - risk_label_accuracy: 0.3360 - risk_label_loss: 40.4062
Test Loss dan Metrics: [587246010368.0, 40.406227111816406, 588196020224.0, 708469.6875, 0.335999995470047]


In [9]:
# Menyimpan Model
model.save('model_asuransi_kendaraan.h5')

import joblib
joblib.dump(scaler, 'scaler_asuransi_kendaraan.pkl')

['scaler_asuransi_kendaraan.pkl']

In [10]:
# Load Model dan Scaler untuk Prediksi

from tensorflow.keras.models import load_model
import joblib

# Load model dan scaler
model = load_model('model_asuransi_kendaraan.h5', compile=False)
scaler = joblib.load('scaler_asuransi_kendaraan.pkl')

# Fungsi prediksi risiko dan premi baru
def prediksi_risiko_premi_kendaraan(umur_kendaraan, harga_kendaraan, pengemudi_muda, area_risiko,
                                    base_premi=150_000):
    input_df = pd.DataFrame({
        'umur_kendaraan': [umur_kendaraan],
        'harga_kendaraan': [harga_kendaraan],
        'pengemudi_muda': [pengemudi_muda],
        'area_risiko': [area_risiko]
    })

    input_scaled = input_df.copy()
    input_scaled[['umur_kendaraan', 'harga_kendaraan']] = scaler.transform(input_df[['umur_kendaraan', 'harga_kendaraan']])

    prediksi = model.predict(input_scaled)
    kelas_risiko = np.argmax(prediksi[0], axis=1)[0]
    premi_final = base_premi + (kelas_risiko * 125_000) + (harga_kendaraan * 0.0035)

    return kelas_risiko, int(premi_final)